<a href="https://colab.research.google.com/github/sanjib7777/-Hyperspectral-Image-Classification/blob/main/Pavia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.io as sio
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def load_pavia_university():
    data = sio.loadmat('PaviaU.mat')['paviaU']
    labels = sio.loadmat('PaviaU_gt.mat')['paviaU_gt']
    return data, labels

def preprocess(data):
    data = data.astype(np.float32)
    data -= np.min(data)
    data /= np.max(data)
    return data

def apply_pca(data, num_components=30):
    h, w, c = data.shape
    reshaped = data.reshape(-1, c)
    pca = PCA(n_components=num_components)
    reduced = pca.fit_transform(reshaped)
    return reduced.reshape(h, w, num_components)

def extract_patches(data, labels, patch_size=7):
    margin = patch_size // 2
    padded_data = np.pad(data, ((margin, margin), (margin, margin), (0, 0)), mode='constant')
    h, w, c = data.shape
    patches, targets, indices = [], [], []
    for i in range(h):
        for j in range(w):
            if labels[i, j] == 0:
                continue
            patch = padded_data[i:i+patch_size, j:j+patch_size, :]
            patches.append(patch)
            targets.append(labels[i, j])
            indices.append((i, j))
    return np.array(patches), np.array(targets), np.array(indices)

In [ ]:
class HSIDataset(Dataset):
    def __init__(self, x_img, x_graph, y):
        self.x_img = x_img
        self.x_graph = x_graph
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.x_img[idx], self.x_graph[idx], self.y[idx]

class SpectralSpatialCNN(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv3d = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.Conv3d(8, 16, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU()
        )
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 7, 7, in_channels)
            out = self.conv3d(dummy)
            self.flattened = out.view(1, -1).shape[1]
        self.fc = nn.Linear(self.flattened, 128)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv3d(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
class GCNBranch(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, 128)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, 0.3, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        return x

class DualBranchFusion(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.branch1 = SpectralSpatialCNN(in_channels)
        self.branch2 = GCNBranch(in_channels)
        self.attn = nn.Sequential(
            nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 2), nn.Softmax(dim=1)
        )
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x_img, x_graph_all, edge_index, batch_indices):
        f1 = self.branch1(x_img)
        f2_all = self.branch2(x_graph_all, edge_index)
        f2 = f2_all[batch_indices]
        combined = torch.stack((f1, f2), dim=1)
        attn_weights = self.attn(torch.cat((f1, f2), dim=1)).unsqueeze(2)
        fused = (combined * attn_weights).sum(dim=1)
        return self.classifier(fused)


In [ ]:
if __name__ == '__main__':
    data, labels = load_pavia_university()
    data = preprocess(data)
    data = apply_pca(data, num_components=30)
    patches, targets, _ = extract_patches(data, labels, patch_size=7)

    num_samples = patches.shape[0]
    le = LabelEncoder()
    targets = le.fit_transform(targets)

    x_graph = patches[:, patches.shape[1] // 2, patches.shape[2] // 2, :]
    scaler = StandardScaler()
    x_graph = torch.tensor(scaler.fit_transform(x_graph), dtype=torch.float32).to(device)

    adj = kneighbors_graph(x_graph.cpu().numpy(), n_neighbors=8, mode='connectivity')
    edge_index, _ = dense_to_sparse(torch.tensor(adj.toarray(), dtype=torch.float))
    edge_index = edge_index.to(device).long()


    x_img = torch.tensor(patches, dtype=torch.float32).to(device)
    y = torch.tensor(targets, dtype=torch.long).to(device)
    train_idx, test_idx = train_test_split(np.arange(num_samples), test_size=0.3, random_state=42, stratify=targets)
    train_dataset = HSIDataset(x_img[train_idx], x_graph[train_idx], y[train_idx])
    test_dataset = HSIDataset(x_img[test_idx], x_graph[test_idx], y[test_idx])

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64)

    model = DualBranchFusion(in_channels=30, num_classes=len(np.unique(targets))).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
    criterion = nn.CrossEntropyLoss()

    x_graph_all = x_graph
    model.train()
    for epoch in range(50):
        total_loss = 0
        for x_img_batch, xg_batch, y_batch in train_loader:
            x_img_batch, xg_batch, y_batch = x_img_batch.to(device), xg_batch.to(device), y_batch.to(device)
            batch_indices = torch.where((xg_batch[:, None] == x_graph_all).all(dim=2))[1]
            optimizer.zero_grad()
            outputs = model(x_img_batch, x_graph_all, edge_index, batch_indices)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 99.5266
Epoch 2, Loss: 12.4193
Epoch 3, Loss: 6.7023
Epoch 4, Loss: 4.7907
Epoch 5, Loss: 12.4503
Epoch 6, Loss: 2.3751
Epoch 7, Loss: 0.1471
Epoch 8, Loss: 0.0685
Epoch 9, Loss: 0.0465
Epoch 10, Loss: 0.0245
Epoch 11, Loss: 0.0179
Epoch 12, Loss: 21.9294
Epoch 13, Loss: 2.6667
Epoch 14, Loss: 0.7789
Epoch 15, Loss: 0.4693
Epoch 16, Loss: 1.1668
Epoch 17, Loss: 9.0933
Epoch 18, Loss: 1.2520
Epoch 19, Loss: 1.3619
Epoch 20, Loss: 1.1873
Epoch 21, Loss: 0.1147
Epoch 22, Loss: 0.0207
Epoch 23, Loss: 0.0098
Epoch 24, Loss: 0.0060
Epoch 25, Loss: 0.0053
Epoch 26, Loss: 0.0040
Epoch 27, Loss: 0.0030
Epoch 28, Loss: 0.0035
Epoch 29, Loss: 0.0020
Epoch 30, Loss: 0.0017
Epoch 31, Loss: 0.0011
Epoch 32, Loss: 0.0012
Epoch 33, Loss: 0.0015
Epoch 34, Loss: 0.0014
Epoch 35, Loss: 0.0005
Epoch 36, Loss: 0.0003
Epoch 37, Loss: 8.4614
Epoch 38, Loss: 1.7934
Epoch 39, Loss: 0.1433
Epoch 40, Loss: 0.0556
Epoch 41, Loss: 0.0197
Epoch 42, Loss: 0.0058
Epoch 43, Loss: 0.0062
Epoch 44, Loss: 

In [ ]:
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x_img_batch, xg_batch, y_batch in test_loader:
            x_img_batch, xg_batch, y_batch = x_img_batch.to(device), xg_batch.to(device), y_batch.to(device)
            batch_indices = torch.where((xg_batch[:, None] == x_graph_all).all(dim=2))[1]
            outputs = model(x_img_batch, x_graph_all, edge_index, batch_indices)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    oa = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    kappa = cohen_kappa_score(all_labels, all_preds)

    print(f"\nOverall Accuracy (OA): {oa:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")


Overall Accuracy (OA): 0.9994
F1 Score (Macro): 0.9990
Cohen's Kappa: 0.9992


In [ ]:
torch.save(model.state_dict(), "dual_branch_hsi_model.pth")
print("Model saved to dual_branch_hsi_model.pth")

Model saved to dual_branch_hsi_model.pth
